# Using an example MEDS tool, ACES for labeling

In [19]:
#@title Install ACES


!pip install es-aces
!cd .. && pip install -e . # pip installs meds dev

Obtaining file:///storage/nassim/projects/MEDS-DEV
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for MEDS_DEV (pyproject.toml) ... done
  Created wheel for MEDS_DEV: filename=MEDS_DEV-0.1.dev159+g685a3e6-0.editable-py3-none-any.whl size=5217 sha256=02557ddafb9ef3654b379ce955ebc9b25abdf44395ec08f292260b0b8c05c422
  Stored in directory: /tmp/pip-ephem-wheel-cache-2m4br255/wheels/f3/cd/e7/d6c66344946b3cec8631003801f957ed99735fb1b583af99ba
Successfully built MEDS_DEV
  Attempting uninstall: MEDS_DEV
    Found existing installation: MEDS_DEV 0.1.dev159+g685a3e6
    Uninstalling MEDS_DEV-0.1.dev159+g685a3e6:
      Successfully uninstalled MEDS_DEV-0.1.dev159+g685a3e6


In [20]:
#@title Download E-ICU demo
import os
from pathlib import Path
notebook_dir = os.getcwd()

# Choose MIMICIV or eicu
ROOT_DIR=f"{notebook_dir}/work_dir/mimiciv_demo/"
# ROOT_DIR=f"{notebook_dir}/work_dir/eicu_demo/"
Path(ROOT_DIR).mkdir(parents=True, exist_ok=True)

!echo {ROOT_DIR}

/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/


In [21]:
# From the ACES documentation

task_config = """
description: >-
  This file specifies the base configuration for the prediction of a hospital los being greater than 3days,
  leveraging only the first 48 hours of data after admission, with a 24 hour gap between the input window
  and the target window. Patients who die or are discharged in the gap window are excluded. Note that this
  task is in-**hospital** los, not in-**ICU** los which is a different task.

predicates:
  hospital_admission:
    code: {regex: "HOSPITAL_ADMISSION//.*"}
  hospital_discharge:
    code: {regex: "HOSPITAL_DISCHARGE//.*"}
  death:
    code: MEDS_DEATH
  discharge_or_death:
    expr: or(hospital_discharge, death)

trigger: hospital_admission

windows:
  input:
    start: NULL
    end: trigger + 48h
    start_inclusive: True
    end_inclusive: True
    index_timestamp: end
  gap:
    start: input.end
    end: start + 24h
    start_inclusive: False
    end_inclusive: True
    has:
      hospital_admission: (None, 0)
      discharge_or_death: (None, 0)
  target:
    start: trigger
    end: start + 3d
    start_inclusive: False
    end_inclusive: True
    label: discharge_or_death
"""
MEDS_DIR = ROOT_DIR + "/meds"
TASK_DIR = MEDS_DIR + "/task_labels"
TASK_NAME = "los_in_hospital_first_48h"
TASK_CONFIG_FP = f"{TASK_DIR}/{TASK_NAME}.yaml"
!mkdir -p {TASK_DIR}/{TASK_NAME}
with open(TASK_CONFIG_FP, 'w') as f:
    f.write(task_config)

In [22]:
!aces-cli --multirun data=sharded data.standard=meds data.root={MEDS_DIR}/data data.shard=$(expand_shards  {MEDS_DIR}/data/) cohort_dir={TASK_DIR} cohort_name={TASK_NAME} config_path={TASK_CONFIG_FP}

[2024-12-14 23:34:59,188][HYDRA] Launching 3 jobs locally
[2024-12-14 23:34:59,188][HYDRA] 	#0 : data=sharded data.standard=meds data.root=/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds/data data.shard=held_out/0 cohort_dir=/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds/task_labels cohort_name=los_in_hospital_first_48h config_path=/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds/task_labels/los_in_hospital_first_48h.yaml
2024-12-14 23:34:59.390 | INFO     | aces.__main__:main:149 - Loading config from '/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds/task_labels/los_in_hospital_first_48h.yaml'
2024-12-14 23:34:59.394 | INFO     | aces.config:load:1341 - Parsing windows...
2024-12-14 23:34:59.394 | INFO     | aces.config:load:1350 - Parsing trigger event...
2024-12-14 23:34:59.395 | INFO     | aces.config:load:1392 - Parsing predicates...
2024-12-14 23:34:59.396 | INFO     | aces.__main__:main:159 - Attempting t

In [23]:
import polars as pl

# execute query and get results
df = pl.read_parquet(f"{TASK_DIR}/{TASK_NAME}/**/*.parquet")

print("train prevalence: " + str(round(pl.read_parquet(f"{TASK_DIR}/{TASK_NAME}/train/*.parquet")['boolean_value'].mean(), 3)))
print("tuning prevalence: " + str(round(pl.read_parquet(f"{TASK_DIR}/{TASK_NAME}/tuning/*.parquet")['boolean_value'].mean(), 3)))
print("held_out prevalence: " + str(round(pl.read_parquet(f"{TASK_DIR}/{TASK_NAME}/held_out/*.parquet")['boolean_value'].mean(), 3)))


df.sort('boolean_value')

train prevalence: 0.241
tuning prevalence: 0.169
held_out prevalence: 0.256


subject_id,prediction_time,boolean_value,integer_value,float_value,categorical_value
i64,datetime[μs],bool,i64,f64,str
10004457,2141-12-19 11:00:00,false,null,null,null
10009049,2174-05-28 08:21:00,false,null,null,null
10012853,2175-04-07 15:36:00,false,null,null,null
10012853,2176-11-27 21:28:00,false,null,null,null
10014729,2125-03-01 07:15:00,false,null,null,null
…,…,…,…,…,…
10026406,2129-01-05 15:55:00,true,null,null,null
10035631,2113-07-19 17:15:00,true,null,null,null
10039708,2140-09-27 04:17:00,true,null,null,null


### MEDS-DEV Has tons of pre-defined tasks we can use!!!

In [24]:
TASK_NAME="mortality/in_icu/first_24h"
!../src/MEDS_DEV/helpers/extract_task.sh {MEDS_DIR} "MIMIC-IV" {TASK_NAME}

Running task mortality/in_icu/first_24h on dataset MIMIC-IV with MEDS_ROOT_DIR=/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds and SHARDS=held_out/0,train/0,tuning/0
[2024-12-14 23:35:02,269][HYDRA] Launching 3 jobs locally
[2024-12-14 23:35:02,269][HYDRA] 	#0 : data.shard=held_out/0
2024-12-14 23:35:02.471 | INFO     | aces.__main__:main:149 - Loading config from '/storage/nassim/projects/MEDS-DEV/src/MEDS_DEV/tasks/criteria/mortality/in_icu/first_24h.yaml'
2024-12-14 23:35:02.473 | INFO     | aces.__main__:main:151 - Overriding predicates and/or demographics from '/storage/nassim/projects/MEDS-DEV/src/MEDS_DEV/datasets/MIMIC-IV/predicates.yaml'
2024-12-14 23:35:02.494 | INFO     | aces.config:load:1341 - Parsing windows...
2024-12-14 23:35:02.494 | INFO     | aces.config:load:1350 - Parsing trigger event...
2024-12-14 23:35:02.494 | INFO     | aces.config:load:1392 - Parsing predicates...
2024-12-14 23:35:02.496 | INFO     | aces.__main__:main:159 - Attempting to g

In [25]:
!echo "{TASK_DIR}/{TASK_NAME}/**/*.parquet"

/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds/task_labels/mortality/in_icu/first_24h/**/*.parquet


In [26]:
import polars as pl


# execute query and get results
df = pl.read_parquet(f"{TASK_DIR}/{TASK_NAME}/**/*.parquet")

print("train prevalence: " + str(round(pl.read_parquet(f"{TASK_DIR}/{TASK_NAME}/train/*.parquet")['boolean_value'].mean(), 3)))
print("tuning prevalence: " + str(round(pl.read_parquet(f"{TASK_DIR}/{TASK_NAME}/tuning/*.parquet")['boolean_value'].mean(), 3)))
print("held_out prevalence: " + str(round(pl.read_parquet(f"{TASK_DIR}/{TASK_NAME}/held_out/*.parquet")['boolean_value'].mean(), 3)))


df.sort('boolean_value')

train prevalence: 0.167
tuning prevalence: 0.091
held_out prevalence: 0.062


subject_id,prediction_time,boolean_value,integer_value,float_value,categorical_value
i64,datetime[μs],bool,i64,f64,str
10012853,2176-11-27 02:34:49,false,null,null,null
10014729,2125-02-28 10:03:08,false,null,null,null
10016742,2178-07-04 22:45:00,false,null,null,null
10016742,2178-07-14 08:16:00,false,null,null,null
10016742,2178-07-23 08:19:00,false,null,null,null
…,…,…,…,…,…
10010471,2155-12-03 20:33:00,true,null,null,null
10015931,2177-03-25 21:48:07,true,null,null,null
10037861,2117-03-15 16:34:58,true,null,null,null
